In [2]:
import pandas as pd
import numpy as np

# Load your dataset
data = pd.read_csv('data.csv')

# Impute missing values using interpolation
data.interpolate(method='linear', limit_direction='forward', inplace=True)

# Create a datetime column
data['datetime'] = pd.to_datetime({
    'year': data['year'],  # Used to create datetime, will be dropped afterward
    'month': data['month'],
    'day': data['day'],
    'hour': data['hour']
})

# Set 'datetime' as the index
data.set_index('datetime', inplace=True)

# Drop the 'year' column as per your requirement
data.drop(['year'], axis=1, inplace=True)

# Extract day of the week and weekend indicator
data['day_of_week'] = data.index.dayofweek
data['is_weekend'] = data['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)

# One-hot encode 'cbwd'
data = pd.get_dummies(data, columns=['cbwd'])

# Define the number of hours ahead to predict
n_hours_ahead = 12  # Change this value to predict different hours ahead

# Shift the 'pm2.5' column to get future values
data['pm2.5_future'] = data['pm2.5'].shift(-n_hours_ahead)

# Drop rows with NaN values resulting from shifting
data.dropna(subset=['pm2.5_future'], inplace=True)

# Create the binary target variable based on 'pm2.5_future'
threshold = 50
data['pm2.5_binary'] = data['pm2.5_future'].apply(lambda x: 1 if x > threshold else 0)

# List to hold DataFrames of lag features
lagged_features = []

# Define the features and lags
lag_features = ['DEWP', 'TEMP', 'PRES', 'Iws', 'Is', 'Ir']
lags = range(1, 25)  # Lags from 1 to 24 hours

# Generate lagged features for specified features
for feature in lag_features + ['pm2.5']:
    # Create a dictionary of lagged series
    lagged_data = {f'{feature}_lag_{lag}': data[feature].shift(lag) for lag in lags}
    # Create a DataFrame from the dictionary
    lagged_df = pd.DataFrame(lagged_data)
    # Append to the list
    lagged_features.append(lagged_df)

# Concatenate all lagged features into a single DataFrame
lagged_features_df = pd.concat(lagged_features, axis=1)

# Concatenate lagged features with the original data
data = pd.concat([data, lagged_features_df], axis=1)

# Drop rows with NaN values resulting from lagging
data.dropna(inplace=True)

# Sort data by datetime
data.sort_index(inplace=True)

# Define the split point (e.g., last 20% of data for testing)
split_fraction = 0.8
split_point = int(len(data) * split_fraction)

# Split the data
train_data = data.iloc[:split_point]
test_data = data.iloc[split_point:]

# Separate features and target
X_train = train_data.drop(['pm2.5', 'pm2.5_future', 'pm2.5_binary'], axis=1)
y_train = train_data['pm2.5_binary']

X_test = test_data.drop(['pm2.5', 'pm2.5_future', 'pm2.5_binary'], axis=1)
y_test = test_data['pm2.5_binary']

from sklearn.preprocessing import StandardScaler

# Identify numeric features to scale
numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler on the training data
X_train_scaled = X_train.copy()
X_train_scaled[numeric_features] = scaler.fit_transform(X_train[numeric_features])

# Apply the scaler to the test data
X_test_scaled = X_test.copy()
X_test_scaled[numeric_features] = scaler.transform(X_test[numeric_features])

# Check class distribution in the training set
from collections import Counter

counter = Counter(y_train)
print(f'Class distribution in training set: {counter}')

# Calculate class weights if imbalance exists
neg, pos = np.bincount(y_train)
total = neg + pos
class_weight = {0: (1 / neg) * (total / 2.0),
                1: (1 / pos) * (total / 2.0)}

print(f'Calculated class weights: {class_weight}')

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

# Define the model architecture
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['Precision', 'Recall', 'AUC'])

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,  # Further split of training data for validation
    shuffle=False,
    epochs=100,
    batch_size=64,
    callbacks=[early_stopping],
    class_weight=class_weight  # Use class weights if imbalance exists
)

# Predict probabilities on the test set
y_pred_prob = model.predict(X_test_scaled)

# Convert probabilities to binary predictions
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

# Evaluate the model
from sklearn.metrics import f1_score, roc_auc_score, classification_report, confusion_matrix

# F1 Score
f1 = f1_score(y_test, y_pred)
print(f'F1 Score: {f1:.4f}')

# AUC-ROC
auc = roc_auc_score(y_test, y_pred_prob)
print(f'AUC-ROC: {auc:.4f}')

# Classification Report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

<ipython-input-2-7de5f46f33e4>:8: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data.interpolate(method='linear', limit_direction='forward', inplace=True)


Class distribution in training set: Counter({1: 21731, 0: 13280})
Calculated class weights: {0: 1.3181852409638555, 1: 0.8055542772997101}


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/data_adapter_utils.py:126: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sample_weight[i] = class_weight.get(int(y[i]), 1.0)


Epoch 1/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - AUC: 0.6677 - Precision: 0.7228 - Recall: 0.6434 - loss: 0.6795 - val_AUC: 0.7389 - val_Precision: 0.7687 - val_Recall: 0.7007 - val_loss: 0.6181
Epoch 2/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - AUC: 0.7515 - Precision: 0.7800 - Recall: 0.7021 - loss: 0.6079 - val_AUC: 0.7659 - val_Precision: 0.7852 - val_Recall: 0.6965 - val_loss: 0.5977
Epoch 3/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - AUC: 0.7847 - Precision: 0.8055 - Recall: 0.7170 - loss: 0.5616 - val_AUC: 0.7772 - val_Precision: 0.7960 - val_Recall: 0.6953 - val_loss: 0.5909
Epoch 4/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - AUC: 0.7895 - Precision: 0.8116 - Recall: 0.7176 - loss: 0.5573 - val_AUC: 0.7817 - val_Precision: 0.7974 - val_Recall: 0.6958 - val_loss: 0.5829
Epoch 5/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - AUC: 0.7974 - Precision: 0.8169 - Recall: 0.7208 - loss: 0.5485 - val_AUC: 0.7839 - val_Precision: 0.7857 - val_Recall: 0.7226 - val_loss: 0.